In [ ]:
CATEGORY = 'track' # artist or track
TIMESPAN = 'medium_term' # short_term, medium_term, long_term
LIMIT = 50 # 1-50; the code can handle more than 50, but the Spotify API, at least currently, only allows 50

In [ ]:
import spotipy

import requests
import math
import os
import json
import time

with open('token.txt', 'r') as f:
    token = f.read()

sp = spotipy.Spotify(auth=token)

In [ ]:
top_items = None

# get top tracks or artists
if CATEGORY == 'artist':
    top_items = sp.current_user_top_artists(limit=LIMIT, time_range=TIMESPAN)
elif CATEGORY == 'track':
    top_items = sp.current_user_top_tracks(limit=LIMIT, time_range=TIMESPAN)

if LIMIT > 50:
    print('\033[33m' + '[WARNING] Spotify API only allows 50 items right now.' + '\033[0m')
    offset = 50
    while offset < LIMIT:
        api_limit = 50
        if offset + 50 > LIMIT:
            api_limit = LIMIT - offset

        if CATEGORY == 'artist':
            top_items['items'] += sp.current_user_top_artists(limit=api_limit, time_range=TIMESPAN, offset=offset)['items']
        elif CATEGORY == 'track':
            top_items['items'] += sp.current_user_top_tracks(limit=api_limit, time_range=TIMESPAN, offset=offset)['items']
        offset += 50

print('\033[32m' + 'Got top items' + '\033[0m')

In [ ]:
# extract image url and id from each item and save it to a list
images = []

if top_items is None or len(top_items['items']) == 0:
    print('\033[91m' + '[Error] No items found' + '\033[0m')
    exit()

if len(top_items['items']) < LIMIT:
    len_items = len(top_items['items'])
    print('\033[93m' + f'[Warning] Not enough items found [Only {len_items} found]' + '\033[0m')

if CATEGORY == 'artist':
    for item in top_items['items']:
        images.append({
            'id': item['id'],
            'name': item['name'],
            'url': item['images'][0]['url']
        })
elif CATEGORY == 'track':
    for item in top_items['items']:
        artists = ''
        for artist in item['artists']:
            artists += artist['name'] + ', '
        artists = artists[:-2]
        images.append({
            'id': item['id'],
            'name': '"' + item['name'] + '"\nby\n' + artists,
            'url': item['album']['images'][0]['url']
        })

print('\033[92m' + '[Success] Found {} images'.format(len(images)) + '\033[0m')

In [ ]:
# download images
if not os.path.exists('images'):
    os.mkdir('images')

if not os.path.exists(f'images/{CATEGORY}s'):
    os.mkdir(f'images/{CATEGORY}s')

# create json file with mapping between id and name
if os.path.exists(f'images/{CATEGORY}s.json'):
    with open(f'images/{CATEGORY}s.json', 'r') as f:
        json_data = json.load(f)
else:
    json_data = {}

for image in images:
    json_data[image['id']] = image['name']

with open(f'images/{CATEGORY}s.json', 'w') as f:
    json.dump(json_data, f)

download_count = 0
for image in images:
    if os.path.exists(f'images/{CATEGORY}s/{image["id"]}.jpg'):
        continue
    r = requests.get(image['url'])
    download_count += 1
    with open(f'images/{CATEGORY}s/{image["id"]}.jpg', 'wb') as f:
        f.write(r.content)
existing_count = len(images) - download_count

print(f'\033[92m' + f'[Success] Downloaded {download_count} new images [{existing_count} were already downloaded]' + '\033[0m')

In [ ]:
with open(f'{CATEGORY}s_{TIMESPAN}_size{LIMIT}.html', 'w') as f:
    f.write('<html>')
    f.write('<head>')

    f.write('<style>')
    f.write('body {background-color: #121212;}')
    f.write('table {margin-left: auto; margin-right: auto;}')
    f.write('table, h1 {margin-top: 30px;}')
    f.write('table {margin-bottom: 30px;}')
    f.write('td {padding: 5px;}')
    f.write('img {border: 2px solid #fff;}')
    f.write('img {border-radius: 33%;}')
    f.write('img:hover {border-radius: 5%;}')
    f.write('img:hover {transition: 1.0s;}')
    f.write('img:hover {transform: scale(3);}')
    f.write('img {z-index: 1;}')
    f.write('img:hover {z-index: 100;}')
    f.write('td:hover {z-index: 100;}')

    f.write('td {position: relative;}')
    f.write('img:hover + span {display: block;}')

    f.write('span {display: none;}')
    f.write('span {position: absolute;}')
    f.write('span {background-color: #121212;}')
    f.write('span {color: #fff;}')
    f.write('span {font-family: Consolas;}')
    f.write('span {font-size: 20px;}')
    f.write('span {padding: 5px;}')
    f.write('span {border-radius: 5px;}')
    # f.write('span {opacity: 0.8;}')
    f.write('span {opacity: 1;}')
    f.write('span {text-align: center;}')
    f.write('span {width: 290;}')
    f.write('span {left: 50%;}')
    f.write('span {transform: translateX(-50%);}')

    # Option 1:
    # f.write('span {bottom: calc(100px + 10px);}')
    # Option 2:
    f.write('span {top: calc(215px);}')
    # increase size of background box
    f.write('span {border: 6px solid #fff;}')
    f.write('span {border-radius: 15px;}')

    f.write('span {z-index: 100;}')
    f.write('span {transition: display 1.5s;}')

    f.write('</style>')

    f.write('</head>')
    f.write('<body>')

    CATEGORY_TITLE = CATEGORY.title()

    mapped_timespans = {
        'short_term': 'Last 4 Weeks',
        'medium_term': 'Last 6 Months',
        'long_term': 'All Time'
    }

    TIMESPAN_TITLE = mapped_timespans[TIMESPAN]
    f.write(f'<h1 style="text-align: center; color: white; font-family: Consolas;">Your Top {CATEGORY_TITLE}s [{TIMESPAN_TITLE}]</h1>')

    f.write('<table>')
    length_of_line = int(math.sqrt(len(images)))
    for x in range(0, length_of_line):
        f.write('<tr>')
        for y in range(0, length_of_line):
            name = images[x*length_of_line+y]['name'].replace('\n', '<br>')
            f.write('<td>')
            f.write(f'<a href="https://open.spotify.com/{CATEGORY}/{images[x*length_of_line+y]["id"]}">')
            f.write(f'<img src="images/{CATEGORY}s/{images[x*length_of_line+y]["id"]}.jpg" width="100" height="100">')
            f.write(f'<span>{name}</span>')
            f.write('</a>')
            f.write('</td>')
            #f.write(f'<td><a href="https://open.spotify.com/{CATEGORY}/{images[x*length_of_line+y]["id"]}" title="{images[x*length_of_line+y]["name"]}"><img src="images/{CATEGORY}s/{images[x*length_of_line+y]["id"]}.jpg" width="100" height="100"></a></td>')
            #f.write(f'<td><a href="https://open.spotify.com/{CATEGORY}/{images[x*length_of_line+y]["id"]}"><img src="images/{CATEGORY}s/{images[x*length_of_line+y]["id"]}.jpg" width="100" height="100"></a></td>')
        f.write('</tr>')
    f.write('</table>')

    f.write('<br>')

    f.write('<style>')
    f.write('.footer {color: white;}')
    f.write('.footer {text-decoration: none;}')
    f.write('.footer {font-family: Consolas;}')
    f.write('.footer {font-size: 20px;}')
    f.write('a.footer {text-decoration: underline;}')

    f.write('.category {font-weight: bold;}')
    
    f.write('.red {color: crimson;}')
    f.write('.green {color: lime;}')
    f.write('.blue {color: cyan;}')

    f.write('a.red:hover {background-color: #3c0000;}')
    f.write('a.green:hover {background-color: #003c00;}')
    f.write('a.blue:hover {background-color: #003c3c;}')

    f.write('a.red:hover {color: white;}')
    f.write('a.green:hover {color: white;}')
    f.write('a.blue:hover {color: white;}')

    f.write('a.red:hover {text-decoration: none;}')
    f.write('a.green:hover {text-decoration: none;}')
    f.write('a.blue:hover {text-decoration: none;}')

    f.write('a.red:hover {transition: 0.3s;}')
    f.write('a.green:hover {transition: 0.3s;}')
    f.write('a.blue:hover {transition: 0.3s;}')

    f.write('a.red:hover {border-radius: 5px;}')
    f.write('a.green:hover {border-radius: 5px;}')
    f.write('a.blue:hover {border-radius: 5px;}')

    f.write('a.red:hover {padding: 5px;}')
    f.write('a.green:hover {padding: 5px;}')
    f.write('a.blue:hover {padding: 5px;}')

    f.write('td.linktd {text-align: center;}')
    f.write('td.linktd {width: 180px;}')
    
    f.write('</style>')

    f.write('<h1 style="text-align: center; color: white; font-family: Consolas;">Other files</h1>')
    
    f.write('<table>')

    f.write('<tr>')
    f.write('<td class="linktd"><div class="footer category">Top Artists</div></td>')
    f.write('<td class="linktd"><a class="footer red" href="artists_short_term_size50.html">Last 4 Weeks</a></td>')
    f.write('<td class="linktd"><a class="footer green" href="artists_medium_term_size50.html">Last 6 Months</a></td>')
    f.write('<td class="linktd"><a class="footer blue" href="artists_long_term_size50.html">All Time</a></td>')
    f.write('</tr>')

    f.write('<tr>')
    f.write('<td class="linktd"><div class="footer category">Top Tracks</div></td>')
    f.write('<td class="linktd"><a class="footer red" href="tracks_short_term_size50.html">Last 4 Weeks</a></td>')
    f.write('<td class="linktd"><a class="footer green" href="tracks_medium_term_size50.html">Last 6 Months</a></td>')
    f.write('<td class="linktd"><a class="footer blue" href="tracks_long_term_size50.html">All Time</a></td>')
    f.write('</tr>')
    f.write('</table>')


    f.write('</body>')
    f.write('</html>')

print('\033[92m' + f'[Success] Done! Created file: {CATEGORY}s_{TIMESPAN}_size{LIMIT}.html' + '\033[0m')


import webbrowser
webbrowser.open(f'{CATEGORY}s_{TIMESPAN}_size{LIMIT}.html')